In [1]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
mp_key = "nPJf5FFhl6DXPpC4NQSxAL5REFwR0nb0" #C'est le clé api de materials project personnel?
mp_id = "mp-23040"

In [3]:
with MPRester(mp_key) as m:
    struct = m.get_structure_by_material_id(mp_id)
    struct_prim = m.get_structure_by_material_id(mp_id)
    conv_struct = SpacegroupAnalyzer(struct_prim).get_conventional_standard_structure()
    ops_sym = SpacegroupAnalyzer(conv_struct).get_space_group_operations()
analyse = SpacegroupAnalyzer(struct)
syms = analyse.get_symmetry_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

## Structure

In [4]:
print(struct) #Données sur la structure
print(ops_sym) # => R-3m (166) spacegroup

view = quick_view(conv_struct, "packed", conventional = True)
display(view)
#view.script('rotate on')

print(conv_struct.sites) #Pour avoir coords pour positions de depart par rapport aux vecteurs (a,b,c)

Full Formula (Ca2 P1 I1)
Reduced Formula: Ca2PI
abc   :   7.828366   7.828366   7.828367
angles:  31.966665  31.966669  31.966669
pbc   :       True       True       True
Sites (4)
  #  SP           a          b         c    magmom
---  ----  --------  ---------  --------  --------
  0  Ca    0.770118   0.770118  0.770118        -0
  1  Ca    0.229882   0.229882  0.229882        -0
  2  P     0.5        0.5       0.5              0
  3  I     0         -0         0                0
R-3m (166) spacegroup


JsmolView(layout=Layout(align_self='stretch', height='400px'))

[PeriodicSite: Ca (2.156, -1.245, 2.303) [0.6667, 0.3333, 0.1035], PeriodicSite: Ca (0.0, 0.0, 5.119) [0.0, 0.0, 0.2299], PeriodicSite: Ca (2.156, 1.245, 9.726) [0.3333, 0.6667, 0.4368], PeriodicSite: Ca (2.156, -1.245, 12.54) [0.6667, 0.3333, 0.5632], PeriodicSite: Ca (0.0, 0.0, 17.15) [0.0, 0.0, 0.7701], PeriodicSite: Ca (2.156, 1.245, 19.96) [0.3333, 0.6667, 0.8965], PeriodicSite: P (2.156, 1.245, 3.711) [0.3333, 0.6667, 0.1667], PeriodicSite: P (4.311, -4.441e-16, 11.13) [1.0, 1.0, 0.5], PeriodicSite: P (2.156, -1.245, 18.56) [0.6667, 0.3333, 0.8333], PeriodicSite: I (0.0, 0.0, 0.0) [0.0, 0.0, 0.0], PeriodicSite: I (2.156, -1.245, 7.422) [0.6667, 0.3333, 0.3333], PeriodicSite: I (2.156, 1.245, 14.84) [0.3333, 0.6667, 0.6667]]


# Symétries du maille

### Symétrie de reflexion (symètrie miroir)

In [5]:
i_atom = 9 #Obtained by hovering over desired atom in (conventional struct) in the viewer!
i_symmop = 8 #Obtained from jsmol crystal symmetry explorer

view = quick_view(struct_prim, "packed", conventional = True)
display(view)
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

pos_depart = conv_struct.sites[0].frac_coords #C'est la coord de l'atome qui corresponds a celle de l'atome Ca au position 9
pos_final = ops_sym[i_symmop-1].operate(pos_depart)

print("Symétrie: reflexion/ symètrie miroir autour plan de normale (1,1,0): (x,y,z) => (-y,-x,z)", "\n")
print(ops_sym[i_symmop-1], "\n")
print("Position initial (par rapport aux vecteurs a,b,c) de l'atome sélectionnée: ", "\n", pos_depart, " => atome de Ca")
print("Position de l'atome après application de l'élément de symétrie: ", "\n", pos_final)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

Symétrie: reflexion/ symètrie miroir autour plan de normale (1,1,0): (x,y,z) => (-y,-x,z) 

Rot:
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.] 

Position initial (par rapport aux vecteurs a,b,c) de l'atome sélectionnée:  
 [0.66666667 0.33333333 0.1034518 ]  => atome de Ca
Position de l'atome après application de l'élément de symétrie:  
 [-0.33333333 -0.66666667  0.1034518 ]


La symètrie de réflexion (plan) est une opération de deuxiéme espéce qui inverse la chiralité, avec la déterminant de la matrice de rotation ayant une déterminant qui vaut -1.

### Symétrie de rotation

In [6]:
i_atom = 2 #Obtained by hovering over desired atom in (conventional struct) in the viewer!
i_symmop = 3 #Obtained from jsmol crystal symmetry explorer

view = quick_view(struct_prim, "packed", conventional = True)
display(view)
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

pos_depart = conv_struct.sites[6].frac_coords #C'est la coord de l'atome qui corresponds a celle de l'atome I au position 
pos_final = ops_sym[i_symmop-1].operate(pos_depart)

print("Symétrie: rotation de degré 3 autour du vecteur c: (x,y,z) => (-y,x-y,z)", "\n")
print(ops_sym[i_symmop-1], "\n")
print("Position initial (par rapport aux vecteurs a,b,c) de l'atome sélectionnée: ", "\n", pos_depart, " => atome de P")
print("Position de l'atome après application de l'élément de symétrie: ", "\n", pos_final)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

Symétrie: rotation de degré 3 autour du vecteur c: (x,y,z) => (-y,x-y,z) 

Rot:
[[ 0. -1.  0.]
 [ 1. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.] 

Position initial (par rapport aux vecteurs a,b,c) de l'atome sélectionnée:  
 [0.33333333 0.66666667 0.16666667]  => atome de P
Position de l'atome après application de l'élément de symétrie:  
 [-0.66666667 -0.33333333  0.16666667]


La rotation de 3éme degré (de pi/3) conserve la chiralité, avec la déterminant de la matrice de rotation ayant une déterminant qui vaut 1.

### Symétrie de rotation => (Que deux symètries uniques pour mon maille)

In [7]:
i_atom = 8 #Obtained by hovering over desired atom in (conventional struct) in the viewer!
i_symmop = 3 #Obtained from jsmol crystal symmetry explorer

view = quick_view(struct_prim, "packed", conventional = True)
display(view)
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

pos_depart = conv_struct.sites[10].frac_coords #C'est la coord de l'atome qui corresponds a celle de l'atome I au position 
pos_final = ops_sym[i_symmop-1].operate(pos_depart)

print("Symétrie: rotation de degré 3 autour du vecteur c: (x,y,z) => (-y,x-y,z)", "\n")
print(ops_sym[i_symmop-1], "\n")
print("Position initial (par rapport aux vecteurs a,b,c) de l'atome sélectionnée: ", "\n", pos_depart, " => atome de I")
print("Position de l'atome après application de l'élément de symétrie: ", "\n", pos_final)

JsmolView(layout=Layout(align_self='stretch', height='400px'))

Symétrie: rotation de degré 3 autour du vecteur c: (x,y,z) => (-y,x-y,z) 

Rot:
[[ 0. -1.  0.]
 [ 1. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.] 

Position initial (par rapport aux vecteurs a,b,c) de l'atome sélectionnée:  
 [0.66666667 0.33333333 0.33333333]  => atome de I
Position de l'atome après application de l'élément de symétrie:  
 [-0.33333333  0.33333333  0.33333333]


La rotation de 3éme degré (de pi/3) conserve la chiralité, avec la déterminant de la matrice de rotation ayant une déterminant qui vaut 1.

In [8]:
#for i in range(len(syms)): #MAUVAIS MATRICES, UTILISER ops_sym!
    #print(i)
    #print(syms[i])
#for i in range(len(ops_sym)): #CELUI A UTILISER - corresponds a jsmol crystal symmetry explorer
    #print(i)
    #print(ops_sym[i])